In [1]:
import sys
import json
sys.path.append('../src')

import requests

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [54]:
ind = {'000000': 'Total nonfarm',
       '110099': 'Mining \& logging',
       '230000': 'Construction',
       '320000': 'Durable goods manufacturing',
       '340000': 'Nondurable goods manufacturing',
       '510000': 'Information',
       '510099': 'Financial activities',
       '540099': 'Professional \& business services',
       '440000': 'Retail trade',
       '480099': 'Transportation, warehousing, \& utilities',
       '610000': 'Educational services',
       '620000': 'Health care \& social services',
       '720000': 'Accommodation \& food services',
       '910000': 'Federal government',
       '920000': 'State \& local government'}

series = {'JTS00000000JOL': 'Openings', 
          'JTS00000000HIL': 'Hires', 
          'JTS00000000QUL': 'Quits',
          'JTS00000000TSL': 'Separations',
          'LNS13000000': 'Unemp'}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'QUR'
    series[i] = i
    
df = bls_api(series, (2000, 2020), bls_key)

df.to_csv(data_dir/'jolts_master.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [71]:
series2 = {}
for code, name in ind.items():
    i = 'JTS' + code + '00' + 'JOR'
    series2[i] = i
    
df = bls_api(series2, (2000, 2020), bls_key)

df.to_csv(data_dir/'jolts_master2.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [55]:
df = (pd.read_csv(data_dir / 'jolts_master.csv', parse_dates=['date'])
        .set_index('date'))

levels_srs = ['Openings', 'Hires', 'Quits', 'Separations', 'Unemp']
rates_srs = ['JTS72000000QUR', 'JTS00000000QUR']

levels_data = df[levels_srs].dropna() / 1000
levels_data.to_csv(data_dir / 'jolts.csv', index_label='date')

ltdata = levels_data.iloc[-1]

ltdate = dtxt(ltdata.name)['mon1']
write_txt(text_dir / 'jolts_ltdate.txt', ltdate)
ltopen = ltdata['Openings']
lthire = ltdata['Hires']
ltquit = ltdata['Quits']
ltsep = ltdata['Separations']
ltun = ltdata['Unemp']

ratio = ltdata['Openings'] / ltdata['Unemp']
ratiop = levels_data['Openings'].iloc[-12] / levels_data['Unemp'].iloc[-12]
p3date = dtxt(levels_data.index[-37])['mon1']
ratio3 = levels_data['Openings'].iloc[-37] / levels_data['Unemp'].iloc[-37]

text = (f'In {ltdate}, there were {ltopen:.1f} million total job openings '+
        f'and {lthire:.1f} million hires completed. In the same month '+
        f'there were {ltsep:.1f} million total separations, of '+
        f'which {ltquit:.1f} million were voluntary. In comparison, '+
        f'there are {ltun:.1f} million unemployed persons in {ltdate}. The ratio '+
        f'of job openings to unemployed persons was {ratio:.1f} in the latest '+
        f'month, compared to {ratiop:.1f} in the same month one year prior, and '+
        f'{ratio3:.1f} in {p3date}.')

write_txt(text_dir / 'jolts2.txt', text)

text

'In December 2019, there were 6.4 million total job openings and 5.9 million hires completed. In the same month there were 5.7 million total separations, of which 3.5 million were voluntary. In comparison, there are 5.8 million unemployed persons in December 2019. The ratio of job openings to unemployed persons was 1.1 in the latest month, compared to 1.2 in the same month one year prior, and 0.8 in December 2016.'

In [56]:
rates_data = df[rates_srs].dropna().rename({'JTS00000000QUR': 'TOT_QU', 'JTS72000000QUR': 'AFS_QU'}, axis=1)

rates_data.to_csv(data_dir / 'quits.csv', index_label='date')

ltdata = rates_data.iloc[-1]

afs_max = rates_data['AFS_QU'].max()
afs_idxmax = dtxt(rates_data['AFS_QU'].idxmax())['mon1']

text = (f'In {ltdate}, the total quits rate in all industries was {ltdata.TOT_QU} percent. '+
        f'The accommodations and food services quits rate was {ltdata.AFS_QU} '+
        f'percent; the series high for the industry group was {afs_max} percent in {afs_idxmax}.')

write_txt(text_dir / 'quits_afs.txt', text)

text

'In December 2019, the total quits rate in all industries was 2.3 percent. The accommodations and food services quits rate was 4.9 percent; the series high for the industry group was 6.3 percent in January 2001.'

In [67]:
max_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().max()
min_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().min()
lt_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().iloc[-1]

qur = pd.DataFrame({'max': max_qur, 'min': min_qur, 'latest': lt_qur})

quits = {}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'QUR'
    quits[i] = name
    
qur.index = qur.index.map(quits)
qur['width'] = qur['max'] - qur['min']
qur.index.name = 'name'
final = qur[['min', 'width', 'latest']].sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.13 for i in list(range(0, len(qur)))]
final.to_csv(data_dir / 'quits_ind.csv', sep=';')

In [72]:
df = (pd.read_csv(data_dir / 'jolts_master2.csv', parse_dates=['date'])
        .set_index('date'))

max_qur = df.loc[:,df.columns.str.endswith('JOR')].dropna().max()
min_qur = df.loc[:,df.columns.str.endswith('JOR')].dropna().min()
lt_qur = df.loc[:,df.columns.str.endswith('JOR')].dropna().iloc[-1]

qur = pd.DataFrame({'max': max_qur, 'min': min_qur, 'latest': lt_qur})

quits = {}

for code, name in ind.items():
    i = 'JTS' + code + '00' + 'JOR'
    quits[i] = name
    
qur.index = qur.index.map(quits)
qur['width'] = qur['max'] - qur['min']
qur.index.name = 'name'
final = qur[['min', 'width', 'latest']].sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.13 for i in list(range(0, len(qur)))]
final.to_csv(data_dir / 'openings_ind.csv', sep=';')